The code is from https://github.com/HarshSulakhe/3DMNIST-PyTorch

# Import Packages

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *
import h5py
from utils.plot3D import *
from utils.utils import *

# Load the vectors 

In [2]:
with h5py.File('/home/asabuncuoglu13/dataset/mnist3d/full_dataset_vectors.h5', "r") as hf:    
    X_train = hf["X_train"][:]
    targets_train = hf["y_train"][:]
    X_test = hf["X_test"][:] 
    targets_test = hf["y_test"][:]
    
X_train = rgb_data_transform(X_train)
X_test = rgb_data_transform(X_test)

# Create Datasets and Dataloaders

In [3]:
train_x = torch.from_numpy(X_train).float()
train_y = torch.from_numpy(targets_train).long()
test_x = torch.from_numpy(X_test).float()
test_y = torch.from_numpy(targets_test).long()

batch_size = 2000

train_ds = torch.utils.data.TensorDataset(train_x,train_y)
test_ds = torch.utils.data.TensorDataset(test_x,test_y)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size = batch_size, shuffle = False)

# CNN and Linear Models

In [4]:
class CNNModel(nn.Module):

    def __init__(self):
        super(CNNModel,self).__init__()
        self.conv1 = nn.Conv3d(3,8,3)
        self.conv2 = nn.Conv3d(8,32,3)
        self.conv3 = nn.Conv3d(32,32,3)
        # self.conv4 = nn.Conv3d(64,128,3)
        self.relu = nn.ReLU()
        self.pool1 = nn.MaxPool3d(4)
        # self.pool2 = nn.MaxPool3d(2)
        # self.linear1 = nn.Linear(128*8,128)
        self.linear2 = nn.Linear(32,10)
        self.batchnorm = nn.BatchNorm3d(32)
        self.dropout = nn.Dropout(0.5)

    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.conv3(x)
        # x = self.conv4(x)
        # x = self.pool1(x)
        x = self.batchnorm(x)
        x = x.view(x.size()[0],-1)
        # x = self.linear1(x)
        # x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

class LinearModel(nn.Module):

    def __init__(self):
        super(LinearModel,self).__init__()
        self.l1 = nn.Linear(4096,1024)
        self.l2 = nn.Linear(1024,256)
        self.l3 = nn.Linear(256,64)
        self.l4 = nn.Linear(64,10)
        self.elu = nn.ELU()
        self.dropout = nn.Dropout(0.5)

    def forward(self,x):
        x = x.view(-1,4096)
        x = self.l1(x)
        x = self.elu(x)
        x = self.dropout(x)
        x = self.l2(x)
        x = self.elu(x)
        x = self.dropout(x)
        x = self.l3(x)
        x = self.elu(x)
        x = self.l4(x)
        return x

# Define Hyperparameters

In [5]:
num_epochs = 160

model = CNNModel()
# model = LinearModel()
model = model.cuda()

error = nn.CrossEntropyLoss()

learning_rate = 0.00005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer,0.005,0.1,cycle_momentum=False)

# Train the Model

In [7]:
train_loss_list = []
test_loss_list = []
train_accuracy_list = []
test_accuracy_list = []
for epoch in range(num_epochs):
    model.train()
    running_acc = 0
    train_loss = 0
    for i, (images, labels) in enumerate(train_loader):     
        train = Variable(images.permute(0,4,1,2,3)).cuda()
        labels = Variable(labels).cuda()
        optimizer.zero_grad()
        outputs = model(train)
        loss = error(outputs, labels)
        train_loss+=loss.data
        train_accuracy = accuracy_score(torch.argmax(outputs,axis = 1).cpu().numpy(),labels.cpu().numpy())
        running_acc+=train_accuracy*len(images)
        loss.backward()
        optimizer.step()
#         scheduler.step()
    running_acc/=len(train_ds)
    train_loss/=5
    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            test = Variable(images.permute(0,4,1,2,3)).cuda()
            outputs = model(test).detach()
            test_loss = error(outputs,labels.cuda())
            test_accuracy = accuracy_score(torch.argmax(outputs,axis = 1).cpu().numpy(),labels.cpu().numpy())
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss.data)
    test_accuracy_list.append(test_accuracy)
    train_accuracy_list.append(train_accuracy)
    

    print('Epoch: {}  Train Loss:{:.2f}, Test Loss:{:.2f}, Train Accuracy:{:.2f}, Test Accuracy:{:.2f}'.format(epoch, train_loss,test_loss,train_accuracy,test_accuracy))


Epoch: 0  Train Loss:2.56, Test Loss:2.30, Train Accuracy:0.12, Test Accuracy:0.09
Epoch: 1  Train Loss:2.46, Test Loss:2.30, Train Accuracy:0.15, Test Accuracy:0.11
Epoch: 2  Train Loss:2.40, Test Loss:2.29, Train Accuracy:0.15, Test Accuracy:0.12
Epoch: 3  Train Loss:2.34, Test Loss:2.28, Train Accuracy:0.18, Test Accuracy:0.13
Epoch: 4  Train Loss:2.26, Test Loss:2.27, Train Accuracy:0.21, Test Accuracy:0.16
Epoch: 5  Train Loss:2.22, Test Loss:2.25, Train Accuracy:0.26, Test Accuracy:0.20
Epoch: 6  Train Loss:2.17, Test Loss:2.23, Train Accuracy:0.25, Test Accuracy:0.25
Epoch: 7  Train Loss:2.13, Test Loss:2.19, Train Accuracy:0.27, Test Accuracy:0.29
Epoch: 8  Train Loss:2.09, Test Loss:2.16, Train Accuracy:0.31, Test Accuracy:0.32
Epoch: 9  Train Loss:2.06, Test Loss:2.11, Train Accuracy:0.31, Test Accuracy:0.35
Epoch: 10  Train Loss:2.02, Test Loss:2.06, Train Accuracy:0.34, Test Accuracy:0.38
Epoch: 11  Train Loss:1.99, Test Loss:2.00, Train Accuracy:0.35, Test Accuracy:0.39
Ep

KeyboardInterrupt: 

# Test the Model

In [ ]:
model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        test = Variable(images.permute(0,4,1,2,3)).cuda()
        outputs = model(test).detach()
        test_loss = error(outputs,labels.cuda())
        test_accuracy = accuracy_score(torch.argmax(outputs,axis = 1).cpu().numpy(),labels.cpu().numpy())
print(test_accuracy)
torch.save(model.state_dict,'./CNN_5e-5.pth')

# Generate Plots

In [ ]:
plt.plot(train_loss_list,label = 'train')
plt.plot(test_loss_list,label = 'test')
plt.legend()
plt.xlabel("Number of epochs")
plt.ylabel("Loss")
plt.title("Loss vs Number of epochs")
plt.show()

In [ ]:
plt.plot(train_accuracy_list,color = "red",label = 'train')
plt.plot(test_accuracy_list,color = "green",label = 'test')
plt.legend()
plt.xlabel("Number of epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy vs Number of epochs")
plt.show()